In [1]:
import json
import glob
import h_gen
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import utils
import importlib
import meta
metadata = utils.load("./variables/metadata.p")
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
import os
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.applications.vgg16 import VGG16
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
from shutil import copyfile
importlib.reload(utils)
importlib.reload(meta)

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmp857z_yum/m91973e5c136ea49268a916ff971b7377.lib and object C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmp857z_yum/m91973e5c136ea49268a916ff971b7377.exp

Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)
C:\Program Files\Anaconda3\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


<module 'meta' from 'C:\\Users\\yolo\\Desktop\\fish\\meta.py'>

First, we need to create a dataset. 

In [2]:
# this generates the rectangles to crop.
# size is the average size of rectangles to crop
# there is also a variation in the size that can be tuned
def rectangle_generator(height, width, size=150, variation=0.3, rectangles=[]):
    xs = np.linspace(0, width, 8)
    ys = np.linspace(0, height, 8)
    for x in xs[1:-1]:
        for y in ys[1:-1]:
            sides = np.random.uniform(size*(1-variation), size*(1+variation), (2,))
            #print("x=" + str(x) + "   y=" + str(y))
            rect = (int(x-sides[0]/2), int(y-sides[1]/2), int(x+sides[0]/2), int(y+sides[1]/2))
            overlapped = False
            for rectangle in rectangles:
                overlapped = meta.overlap(rectangle, rect)
                if overlapped:
                    break
            if not overlapped:
                yield rect

In [3]:
# We create the folders
h_gen.mk("cropped")
for key, value in meta.classes_dictionary.items():
    h_gen.mk("cropped/" + key)

# We crop the images
for key, value in tq(metadata.items()):
    
    img = Image.open(value["path"])
    rectangles = []
    # First, we crop the fishes
    if "rectangles" in value:
        rectangles = value["rectangles"]
        for i, rect in enumerate(rectangles):
            path_to_save = "cropped/" + value["class"] + "/" + value["filename"][:-4] + "_" + str(i) + value["filename"][-4:]
            
            # We want the fishes to be in squares, so we need to find the center
            side = max(rect["width"], rect["height"])
            x = rect["x"] + rect["width"]/2
            y = rect["y"] + rect["height"]/2
            m = 15
            
            cropped = img.crop((x-side/2-m , y-side/2-m, x+side/2+m , y+side/2+m))
            cropped.save(path_to_save)
    
    # Now we do crop of the background
    for i, rect in enumerate(rectangle_generator(value["width"], value["height"], 230, 0.3, rectangles)):
        cropped = img.crop(rect)
        path_to_save = "cropped/NoF/" + value["filename"][:-4] + "_" + str(i) + value["filename"][-4:]
        cropped.save(path_to_save)

In [4]:
def split_train_test(nb_images, train_dir):
    temp = train_dir.find("train")
    test_dir = train_dir[:temp] + "test" + train_dir[temp+5:]
    
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        
    classes = os.listdir(train_dir)
        
    train_dir = [train_dir + x + "/" for x in classes]
    test_dir = [test_dir + x + "/" for x in classes]
    
    for folder in test_dir:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    for train_class, test_class in zip(train_dir, test_dir):
        files = os.listdir(train_class)[:nb_images]
        for file in files:
            copyfile(train_class + file,test_class + file)
            os.remove(train_class + file)

In [5]:
split_train_test(30,"cropped/")

In [2]:
train_datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.01,
    height_shift_range=0.01,
    shear_range=0.0005,
    horizontal_flip=True,
    vertical_flip=True)
test_datagen = ImageDataGenerator()

In [3]:
target_size=(224,224)
train_generator = train_datagen.flow_from_directory('cropped_train_gen', batch_size=32, 
                                                    target_size=target_size, preprocess_input=preprocess_input)
validation_generator = test_datagen.flow_from_directory('cropped_test', batch_size=32, target_size=target_size
                                                        , preprocess_input=preprocess_input)

Found 79794 images belonging to 8 classes.
Found 240 images belonging to 8 classes.


In [4]:
out = 64
base = VGG16(include_top=False)
img_input = Input(shape=(3,None,None))
x = base(img_input)
x =Convolution2D(out, 3,3, activation="relu", border_mode="same")(x)
x =Convolution2D(out, 3,3, activation="relu", border_mode="same")(x)
x =Convolution2D(out, 3,3, activation="relu", border_mode="same")(x)
x =Convolution2D(8, 3,3, border_mode="same")(x)
x = GlobalAveragePooling2D()(x)
x = Activation("softmax")(x)
model = Model(img_input, x)
for layer in base.layers:
    layer.trainable=False

base.trainable=False

In [5]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
vgg16 (Model)                    (None, 512, None, Non 14714688    input_2[0][0]                    
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, None, None 294976      vgg16[1][0]                      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, None, None 36928       convolution2d_1[0][0]            
___________________________________________________________________________________________

In [6]:
opt = SGD(lr=0.01, decay=0.01)
#opt = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy')

In [7]:
# train the model on the new data for a few epochs
model.fit_generator(train_generator,
        samples_per_epoch=1024,
        nb_epoch=100,
        validation_data=validation_generator,
        verbose=0,
        nb_val_samples=240, callbacks=[remote])

Exception in thread Thread-46:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\Anaconda3\lib\site-packages\keras\engine\training.py", line 409, in data_generator_task
    generator_output = next(generator)
  File "C:\Program Files\Anaconda3\lib\site-packages\keras\preprocessing\image.py", line 474, in __next__
    return self.next(*args, **kwargs)
  File "C:\Program Files\Anaconda3\lib\site-packages\keras\preprocessing\image.py", line 637, in next
    a = x.shape()
TypeError: 'tuple' object is not callable



ValueError: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None

In [24]:
def create_new_pictures(train_folder, class_folder, generator,total_images, 
                        batch_size = 32, target_size = (224,224), color_mode = "rgb"):
    #count = len(os.listdir(train_folder + "/" + class_folder))
    count = 0  
    gen_folder = train_folder + "_gen/"
        
    if not os.path.exists(gen_folder):
        os.makedirs(gen_folder)
        
    if not os.path.exists(gen_folder + class_folder):
        os.makedirs(gen_folder + class_folder)
        
    
    progbar = tq(total = total_images, leave=False)
    progbar.update(count)
    
    
    for X in generator.flow_from_directory(train_folder, target_size=target_size,
                                             classes=[class_folder], 
                                             save_to_dir=train_folder + "_gen/" + class_folder, 
                                             batch_size=batch_size, color_mode=color_mode):
        count +=batch_size
        progbar.update(batch_size)
        if count >= total_images:
            break
    progbar.close()
    

In [27]:
def equilibrate_dataset(folder, nb_images_per_class, generator):
    c = 0
    while c < nb_images_per_class:
        for string in os.listdir(folder):
            create_new_pictures(folder, string, generator, 30000, 1, (64,64))
        c += 30000

In [28]:
equilibrate_dataset("cropped_train", 50000, test_datagen)

Found 2500 images belonging to 1 classes.
Found 277 images belonging to 1 classes.
Found 94 images belonging to 1 classes.
Found 71 images belonging to 1 classes.
Found 103961 images belonging to 1 classes.
Found 359 images belonging to 1 classes.
Found 166 images belonging to 1 classes.
Found 793 images belonging to 1 classes.
Found 2500 images belonging to 1 classes.


KeyboardInterrupt: 